# "Conv2d to rnn"

In [1]:
from comet_ml import Experiment
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras.regularizers import l2

Using TensorFlow backend.


### Load Data

In [2]:
data_path='../preprocessing/preprocessed_data'

In [3]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=True)
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [4]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=5),
             keras.callbacks.ModelCheckpoint('conv2d_to_rnn.h5', monitor='val_loss', save_best_only=True)]

### Simple First Model

In [11]:
weight_decay = 0.01

model1 = models.Sequential()
model1.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=32,
                         fmin=0.0, fmax=5000, power_melgram=1.0,
                         return_decibel_melgram=True, trainable_fb=False,
                         trainable_kernel=False))
model1.add(Normalization2D(str_axis='data_sample'))

model1.add(layers.Conv2D(16, (4, 4), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model1.add(layers.MaxPooling2D((2, 2 )))
model1.add(layers.BatchNormalization())

model1.add(layers.Conv2D(32, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model1.add(layers.MaxPooling2D((2, 1)))
model1.add(layers.BatchNormalization())

model1.add(layers.Conv2D(32, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model1.add(layers.MaxPooling2D((2, 1)))
model1.add(layers.BatchNormalization())

model1.add(layers.Conv2D(64, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model1.add(layers.MaxPooling2D((2, 1)))
model1.add(layers.BatchNormalization())

model1.add(layers.Conv2D(64, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model1.add(layers.MaxPooling2D((2, 1)))
model1.add(layers.BatchNormalization())

model1.add(layers.Reshape((-1, 64)))
model1.add(layers.GRU(64, dropout=0.3))
model1.add(layers.Dense(3, activation='softmax'))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_2 (Melspectro (None, 32, 313, 1)        271392    
_________________________________________________________________
normalization2d_2 (Normaliza (None, 32, 313, 1)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 313, 16)       272       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 156, 16)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 156, 16)       64        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 156, 32)       2080      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 156, 32)        0         
__________

=> 94%

### Model 2
derived from https://github.com/YerevaNN/Spoken-language-identification/blob/master/theano/networks/tc_net_rnn_onernn.py

In [ ]:
shared_GRU = layers.CuDNNGRU(15)
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=223, n_hop=256,
                         fmin=0.0, fmax=5500, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.Conv2D(16, (7, 7), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3 ), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Permute((3,2,1)))
model.add(layers.TimeDistributed(shared_GRU))
model.add(layers.Reshape((-1,)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation='softmax'))
#model.summary()

=> 91%

### Model 3: Inception
derived from https://github.com/HPI-DeepLearning/crnn-lid/blob/master/keras/models/inceptionv3_crnn.py

In [42]:
from keras.applications.inception_v3 import InceptionV3

In [68]:
input_tensor = layers.Input(shape=(1, 80000))
x = Melspectrogram(n_dft=512,
                   padding='same', sr=16000, n_mels=128, n_hop=256,
                   fmin=0.0, fmax=5500, power_melgram=1.0,
                   return_decibel_melgram=False, trainable_fb=False,
                   trainable_kernel=False)(input_tensor)

inception_model = InceptionV3(include_top=False, weights=None, input_tensor=x)

x = inception_model.output

#x = GlobalAveragePooling2D()(x)

# (bs, y, x, c) --> (bs, x, y, c)
x = layers.Permute((2, 1, 3))(x)

# (bs, x, y, c) --> (bs, x, y * c)
_a, _y, _c = [int(s) for s in x.shape[1:]]
x = layers.Reshape((_a, _y*_c))(x)
x = layers.wrappers.Bidirectional(layers.LSTM(64, return_sequences=False), merge_mode="concat")(x)
x = layers.Dense(3, activation='softmax')(x)

model = models.Model(input_tensor, outputs=x)
#model.summary()

tooooo biiiigggg...

### Model 3: MobileNet

In [55]:
from keras.applications.mobilenetv2 import MobileNetV2

In [69]:
input_tensor = layers.Input(shape=(1, 80000))
x = Melspectrogram(n_dft=512,
                   padding='same', sr=16000, n_mels=223, n_hop=360,
                   fmin=0.0, fmax=5500, power_melgram=1.0,
                   return_decibel_melgram=False, trainable_fb=False,
                   trainable_kernel=False)(input_tensor)

inception_model = MobileNetV2(include_top=False, weights=None, input_tensor=x,
                            alpha=0.25, pooling=None)

x = inception_model.output

#x = GlobalAveragePooling2D()(x)

# (bs, y, x, c) --> (bs, x, y, c)
x = layers.Permute((2, 1, 3))(x)

# (bs, x, y, c) --> (bs, x, y * c)
_a, _y, _c = [int(s) for s in x.shape[1:]]
x = layers.Reshape((_a, _y*_c))(x)
x = layers.GRU(64, return_sequences=False, dropout=0.5)(x)
x = layers.Dense(3, activation='softmax')(x)

model = models.Model(input_tensor, outputs=x)
#model.summary()

KeyboardInterrupt: 

## Train

In [6]:
experiment = Experiment(api_key="P9qHCZEUF514fowP4zfVDbGBl",
                      project_name="conv-2d-to-rnn", workspace="jotron")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jotron/conv-2d-to-rnn/5c5c18950d264dc1ac0b8a2ff3d67f97



In [12]:
model1.compile(optimizer='Adam',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [13]:
history = model1.fit(x=train_data,
                        y=train_labels,
                        batch_size=64, 
                        epochs=20,
                        validation_data=(val_data, val_labels), 
                        shuffle=True,
                        callbacks=callbacks)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 150s 3ms/step - loss: 1.4429 - acc: 0.4610 - val_loss: 1.1274 - val_acc: 0.5526
Epoch 2/20
50000/50000 [==============================] - 148s 3ms/step - loss: 1.0867 - acc: 0.5612 - val_loss: 1.0241 - val_acc: 0.6121
Epoch 3/20
50000/50000 [==============================] - 149s 3ms/step - loss: 0.8054 - acc: 0.7220 - val_loss: 0.6903 - val_acc: 0.7761
Epoch 4/20
50000/50000 [==============================] - 149s 3ms/step - loss: 0.5654 - acc: 0.8372 - val_loss: 0.5513 - val_acc: 0.8423
Epoch 5/20
50000/50000 [==============================] - 150s 3ms/step - loss: 0.4754 - acc: 0.8784 - val_loss: 0.6278 - val_acc: 0.8103
Epoch 6/20
50000/50000 [==============================] - 149s 3ms/step - loss: 0.4221 - acc: 0.9003 - val_loss: 0.5216 - val_acc: 0.8694
Epoch 7/20
50000/50000 [==============================] - 149s 3ms/step - loss: 0.3954 - acc: 0.9082 - val_loss: 0.4261 - 

In [14]:
experiment.end()

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jotron/conv-2d-to-rnn/5c5c18950d264dc1ac0b8a2ff3d67f97

